In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
data_path = ''
df = pd.read_csv(data_path)

In [4]:
missing_pctg = (df.isnull().sum()/len(df))*100

In [5]:
columns_to_drop = missing_pctg[missing_pctg>95].index
df = df.drop(columns_to_drop, axis = 1)

df = df.drop(['endYear','attributes'], axis = 1)

In [7]:
df = df.drop(['directors','writers'], axis = 1)

df_year = df['startYear'].replace(0,df['startYear'].median())
df['age'] = datetime.now().year - df_year
df['age'] = df['age'].astype(float)
df = df.drop('startYear', axis=1)

In [8]:
num_cols = ['numVotes','runtimeMinutes','isAdult','age','seasonNumber','episodeNumber','ordering','isOriginalTitle']
cat_cols = ['titleType','language']

In [9]:
genres_split = df['genres_x'].str.get_dummies(',')
genres_split = genres_split.fillna('missing')
df = df.drop('genres_x', axis=1)

df_cat = df[cat_cols]
df_cat = df_cat.fillna('missing')
df_encoded = pd.get_dummies(df_cat, columns=cat_cols, prefix=cat_cols)

df = df.drop(df[cat_cols], axis = 1)
df = pd.concat([df,df_encoded], axis = 1)

In [10]:
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

In [11]:
train_target = df['averageRating']
train_target = pd.DataFrame(train_target)

df_pred = df.drop('averageRating', axis = 1)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_pred, train_target, test_size=0.2, random_state=42)

In [13]:
from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(max_depth = 8, random_state=42)
reg.fit(X_train, y_train)
reg.score(X_test, y_test)

0.2282195903374059

In [15]:
import xgboost as xgb 
xgb_model = xgb.XGBRegressor(objective="reg:squarederror",random_state=42)
xgb_model.fit(X_train, y_train)
xgb_model.score(X_test, y_test)

0.27689292487879